From: https://github.com/pytorch/examples/blob/main/mnist/main.py

# Setup

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Network

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

# Train / Test

In [5]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Main

In [14]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=14, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                    help='learning rate (default: 1.0)')
parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                    help='Learning rate step gamma (default: 0.7)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--no-mps', action='store_true', default=False,
                    help='disables macOS GPU training')
parser.add_argument('--dry-run', action='store_true', default=False,
                    help='quickly check a single pass')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')

args = parser.parse_args("--save-model".split())
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

In [8]:
torch.manual_seed(args.seed)

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

device

device(type='cuda')

In [9]:
train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 11180275.73it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 8245571.70it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 9151430.23it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 12889397.00it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [10]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

In [11]:
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.282943
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.296535
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.976175
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.650030
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.442521
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.460979
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.282729
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.436664
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.249838
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.142709
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.231022
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.259762
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.325766
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.140261
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.426044
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.076149
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.271494
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.258434
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.219710
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.136336
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.114919
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.031489
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.067123
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.045952
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.040886
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.078999
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.131843
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.052156
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.021316
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.051192
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.035462
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.023552
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.038743
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.309088
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.069540
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.070211
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.027839
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.207738
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.052507
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.058649


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.042236
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.028834
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.262014
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.003629
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.001223
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.061448
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.048761
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.098145
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.008477
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.137028
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.016247
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.024261
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.013209
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.055434
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.013402
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.092685
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.001827
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.011287
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.029976
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.003425


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.072650
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.018415
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.088330
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.001731
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.009337
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.002824
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.017499
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.023488
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.050330
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.009425
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.070775
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.000645
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.001729
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.010009
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.015035
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.019766
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.015558
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.006500
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.058894
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.120491


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.053700
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.031337

Test set: Average loss: 0.0303, Accuracy: 9905/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.012337
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.006104
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.037226
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.003418
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.013411
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.005098
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.039255
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.019900
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.001442
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.011596
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.021184
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.075779
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.012672
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.005161
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.008361
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.000785
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.046607
Train Epoch: 8 [10

Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.088187
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.001825
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.005117
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.000974
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.074719
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.015819
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.002029
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.008726
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.003732
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.030970
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.004477
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.130333
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.081507
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.047916
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.023384
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.031434
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.016820
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.060499
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.040937
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.045642


Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.011421
Train Epoch: 11 [26240/60000 (44%)]	Loss: 0.023016
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.012441
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.016630
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.017794
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.102098
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.004678
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.038659
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.006682
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.000430
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.016613
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.021510
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.003923
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.000647
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.014252
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.001233
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.049954
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.010328
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.014141
Train Epoch: 11 [37760/60000 (6

Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.025133
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.000893
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.012254
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.003403
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.009136
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.075202
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.005437
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.005637
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.017898
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.021110
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.090091
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.002769
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.036143
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.053762
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.016683
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.024078
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.015982
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.001123
Train Epoch: 13 [19200/60000 (32%)]	Loss: 0.011800
Train Epoch: 13 [19840/60000 (33%)]

Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.075254
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.004186
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.007347
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.012918
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.002940
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.019369
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.011715
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.001316
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.026378
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.021521
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.000526
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.008338
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.024442
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.003544

Test set: Average loss: 0.0275, Accuracy: 9911/10000 (99%)



In [16]:
if args.save_model:
    print('Saving model..')
    torch.save(model.state_dict(), "mnist_cnn.pt")
    print('Done!')

Saving model..
Done!
